#Preprocessing Pipeline

In [ ]:
!wget https://www.dropbox.com/s/opk4pbqoddz8smv/sample_TESS_sim_mag_bins.zip?dl=0 -O sample_sim_mag_bins.zip
#https://www.dropbox.com/s/4m0x95jq1av3w6z/sample_TESS_sim.zip?dl=1 -O sample_sim.zip
print('done')

!ls -l
!7z x sample_sim_mag_bins.zip
import pandas as pd
import numpy as np
import glob

files = glob.glob("./*.csv")
for i in files:
  df = pd.read_csv(i, header=None).values
  np.save(i[:-4]+'.npy',df)
  
!rm *.csv
!wget http://archive.stsci.edu/missions/tess/ete-6/tid/00/000/003/185/tess2019128220341-0000000318565153-0016-s_lc.fits -O sample_tess.fits
!ls -l

In [ ]:
import astropy.io.fits as fits
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from random import shuffle
import astropy.io.fits as fits
from astropy.io.fits.card import UNDEFINED
import batman
from tqdm import tqdm
import pandas as pd
import numpy as np
import glob

def generate_transit_params(n = 100000):
    M = np.linspace(0.3,2.0,n)
    N = 0.287971 * M**-2.35 * 0.001 * 20000
    N = N.astype(int)
    masses = [np.repeat(M[i],N[i]) for i in range(len(N))]
    mass_dist = np.concatenate(masses, axis=0)

    np.random.shuffle(mass_dist)
    mass_dist = mass_dist[:n]
    
    period = np.power(10, np.random.uniform(np.log10(2), np.log10(9), n))
    logRpRs = np.random.uniform(np.log10(20), np.log10(100), n) * -1 
    RpRs = np.power(10, logRpRs)
    t0 = np.random.uniform(0, 1, n)
    b = np.random.uniform(0, 1, n)
    a_vals = np.random.uniform(0.5, 0.7, n)
    aRs = 7.495**(1.0/3.0) *(10**-2) * 214.9394693836 * mass_dist**(4.0/3.0) * period**(2.0/3.0)# assume R ~ M 
    print aRs[0].shape
    print mass_dist[0], period[0], (7.495*(10**-6) * 214.9394693836**3 * mass_dist[0]**3 * period[0]**2 * mass_dist[0]), aRs[0]
    return np.stack((t0,period,RpRs,aRs,b,a_vals, np.zeros_like(a_vals), np.zeros_like(a_vals), np.zeros_like(a_vals)), axis=1)
aRs = generate_transit_params()[:,3]
print aRs.max(), aRs.min()

In [ ]:
def parse_fn(input_data, zeros, gen_params, TransitModel, params):
    params.t0 = gen_params[0] * gen_params[1]
    params.per = gen_params[1]  # orbital period
    params.rp = gen_params[2]  # planet radius (in units of stellar radii)
    params.a = gen_params[3]  # semi-major axis (in units of stellar radii)
    params.inc = 57.2957795 * np.arccos(gen_params[4]/gen_params[3])  # orbital inclination (in degrees)
    params.u = [gen_params[5]] # limb darkening coefficients [u1, u2, u3, u4]
    flux = TransitModel.light_curve(params)
    flux = np.log10(flux) * -2.5
    
    input_data = np.log10(input_data) * -2.5
    
    
    y = np.copy(input_data) * zeros
    mu_y = np.mean(y[y != 0])
    sigma_y = np.std(y[y != 0])
    y = (y - mu_y) / sigma_y
    y *= zeros
    
    x = np.copy(input_data)
    x = (flux + x) * zeros
    mu_x = np.mean(x[x != 0])
    sigma_x = np.std(x[x != 0])
    x = (x - mu_x) / sigma_x
    x *= zeros
    
    qn = np.sum(flux > 0)
    SNR = (qn**0.5) * (gen_params[2]**2 / sigma_x)
    np.reshape(x, (x.shape[0],1))
    np.reshape(y, (y.shape[0],1))
    np.reshape(flux, (flux.shape[0],1))
    return x, y, flux, SNR, mu_x, sigma_x, mu_y, sigma_y


print "Start"
data_count = 9999 * 10
file_count = 9999

middle_count = data_count

gen_parameters = generate_transit_params()
params = batman.TransitParams()  # object to store transit parameters
params.t0 = 0# time of inferior conjunction
params.per = 1  # orbital period
params.rp = 0.05  # planet radius (in units of stellar radii)
params.a = 5  # semi-major axis (in units of stellar radii)
params.inc = 90  # orbital inclination (in degrees)
params.ecc = 0.0  # eccentricity
params.w = 90.  # longitude of periastron (in degrees)
params.limb_dark = "linear"  # limb darkening model
params.u = [0.5] # limb darkening coefficients [u1, u2, u3, u4]

t = np.linspace(0, 28.625, 20610)  # times at which to calculate light curve
m = batman.TransitModel(params, t)  # initializes model

file_id = -1
file = None
header = '' #'D:\\sample_TESS_sim\\'
hdul = fits.open(header + 'sample_tess.fits')

total_features =  np.zeros(dtype='float32', shape=(data_count * 2, 20610, 1))
total_transits =  np.zeros(dtype='float32', shape=(data_count * 2, 20610, 1))
total_y =  np.zeros(dtype='float32', shape=(data_count * 2))
total_params =  np.zeros(dtype='float32', shape=(data_count * 2, 9))
total_SNR =  np.zeros(dtype='float32', shape=(data_count * 2, 1))


files_mag = sorted(glob.glob("./*.csv"))
files = sorted(glob.glob("./*.npy"))
zeros = hdul[1].data['PDCSAP_FLUX']
zeros[zeros != 0] = 1 # this array represents when we have tess data to simulate data gaps due to orbit
for data in tqdm(range(middle_count)):
    if int(data / file_count) != file_id:  # since each file has 9999 stars we'll switch files each time we reach that numer
        file_id = int(data / file_count)
        file = np.load(files[file_id])
        mag_file = pd.read_csv(files_mag[file_id], header=None).values
        print('loading sample_TESS_sim' + str(file_id))
    
    mag = mag_file[data % file_count,-1]
    if not isinstance(mag, float):
        mag = float(mag.replace('..','.'))
    x, y, transit, SNR, mu_x, sigma_x, mu_y, sigma_y = parse_fn(file[data % file_count, :-1], zeros, gen_parameters[data], m, params)  # get the LCs
    gen_parameters[data] = [gen_parameters[data][0], np.log10(gen_parameters[data][1]),gen_parameters[data][2],
                            gen_parameters[data][3], gen_parameters[data][4], gen_parameters[data][5], mu_x, sigma_x, mag]
    #print(total_features[data].shape)
    total_features[data, : , 0] = x
    total_features[data + middle_count, : , 0] = y

    total_transits[data, : , 0] = transit
    
    total_y[data] = 1  # x has injected planet, y dosnt
    total_y[data + middle_count] = 0
    
    total_params[data, :] = gen_parameters[data]
    total_params[data + middle_count, :] = [0]*6 + [mu_y, sigma_y, mag]
    
    total_SNR[data,0] = SNR

print(total_features.shape, total_transits.shape, total_y.shape, total_params.shape, total_SNR.shape)
np.save(header + 'total_x_sim.npy', total_features)
np.save(header + 'total_y_sim.npy', total_y)
np.save(header + 'total_transits_sim.npy', total_transits)
np.save(header + 'total_params_sim.npy', total_params)
np.save(header + 'total_SNR_sim.npy', total_SNR)

total_features = None
total_transits = None
total_y = None
total_params = None
total_SNR = None

#!7z a all_files.zip total_x_sim.npy total_y_sim.npy total_transits_sim.npy total_params_sim.npy total_SNR_sim.npy
print('done')


In [ ]:
import astropy.io.fits as fits
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from random import shuffle
import astropy.io.fits as fits
from astropy.io.fits.card import UNDEFINED
import batman
from tqdm import tqdm
import pandas as pd
import numpy as np
import glob

total_features = None
total_transits = None
total_y = None
total_params = None
total_SNR = None

total_features =  np.load('total_x_sim.npy', mmap_mode =  'r')
total_transits =  np.load('total_transits_sim.npy', mmap_mode =  'r')
total_y =  np.load('total_y_sim.npy')
total_params =  np.load('total_params_sim.npy')
total_SNR =  np.load('total_SNR_sim.npy')

# idxs_train_base_false = np.random.choice(middle_count, size=20000,replace=False)
# idxs_train_base_true = idxs_train_base_false + middle_count

# idxs_train_classification = np.random.choice(len(idxs_train_base_false), size=len(idxs_train_base_false)//2,replace=False)
# idxs_train_classification = np.array([for i,j in zip(idxs_train_base_false, idxs_train_base_true) i if i in idxs_train_classification else j])

# idxs_test_gan = np.array([i for i in range(len(middle_count*2)) if i not in np.concatenate([idxs_train_base, idxs_train_base_true])])

idx = np.random.permutation(middle_count*2)
total_features = total_features[idx]
total_transits = total_transits[idx]
total_y = total_y[idx]
total_params = total_params[idx]
total_SNR = total_SNR[idx]
print('done')

print('start')
print(total_features.shape, total_transits.shape, total_y.shape, total_params.shape, total_SNR.shape)
np.save(header + 'total_x_sim.npy', total_features)
np.save(header + 'total_y_sim.npy', total_y)
np.save(header + 'total_transits_sim.npy', total_transits)
np.save(header + 'total_params_sim.npy', total_params)
np.save(header + 'total_SNR_sim.npy', total_SNR)
np.save(header + 'total_idx_sim.npy', idx)

total_features = None
total_transits = None
total_y = None
total_params = None
total_SNR = None


In [ ]:
#!rm -rf sample_TESS*.npy
#!rm -rf mag_*.csv
#!7z a all_files.zip total_x_sim.npy total_y_sim.npy total_transits_sim.npy total_params_sim.npy total_SNR_sim.npy total_idx_sim.npy

In [ ]:
import astropy.io.fits as fits
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from random import shuffle
import astropy.io.fits as fits
from astropy.io.fits.card import UNDEFINED
import batman
from tqdm import tqdm
import pandas as pd
import numpy as np
import glob

total_features =  np.load('total_x_sim.npy', mmap_mode =  'r')
total_transits =  np.load('total_transits_sim.npy', mmap_mode =  'r')
total_y =  np.load('total_y_sim.npy')
total_params =  np.load('total_params_sim.npy')
total_SNR =  np.load('total_SNR_sim.npy')


total_features_train = total_features[:20000]
total_transits_train = total_transits[:20000]
total_y_train = total_y[:20000]
total_params_train = total_params[:20000]
total_SNR_train = total_SNR[:20000]

total_features_test = total_features[20000:30000]
total_transits_test = total_transits[20000:30000]
total_y_test = total_y[20000:30000]
total_params_test = total_params[20000:30000]
total_SNR_test = total_SNR[20000:30000]
print('done')

print('start')
print(total_features.shape, total_transits.shape, total_y.shape, total_params.shape, total_SNR.shape)
header = "./"
np.save(header + 'total_x_sim_train.npy', total_features_train)
np.save(header + 'total_y_sim_train.npy', total_y_train)
np.save(header + 'total_transit_sim_train.npy', total_transits_train)
np.save(header + 'total_params_sim_train.npy', total_params_train)
np.save(header + 'total_SNR_sim_train.npy', total_SNR_train)


np.save(header + 'total_x_sim_test.npy', total_features_test)
np.save(header + 'total_y_sim_test.npy', total_y_test)
np.save(header + 'total_transit_sim_test.npy', total_transits_test)
np.save(header + 'total_params_sim_test.npy', total_params_test)
np.save(header + 'total_SNR_sim_test.npy', total_SNR_test)

#!7z a all_files_small.zip total_x_sim_train.npy total_y_sim_train.npy total_transit_sim_train.npy total_params_sim_train.npy total_SNR_sim_train.npy total_x_sim_test.npy total_y_sim_test.npy total_transit_sim_test.npy total_params_sim_test.npy total_SNR_sim_test.npy
print('done')


total_features = None
total_transits = None
total_y = None
total_params = None
total_SNR = None

total_features_train = None
total_transits_train = None
total_y_train = None
total_params_train = None
total_SNR_train = None

total_features_test = None
total_transits_test = None
total_y_test = None
total_params_test = None
total_SNR_test = None

In [ ]:
#!rm -rf *_train.npy
#!rm -rf *_test.npy


In [ ]:
import astropy.io.fits as fits
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from random import shuffle
import astropy.io.fits as fits
from astropy.io.fits.card import UNDEFINED
import batman
from tqdm import tqdm
import pandas as pd
import numpy as np
import glob

total_features =  np.load('total_x_sim.npy', mmap_mode =  'r')
total_transits =  np.load('total_transits_sim.npy', mmap_mode =  'r')
total_y =  np.load('total_y_sim.npy')
total_params =  np.load('total_params_sim.npy')
total_SNR =  np.load('total_SNR_sim.npy')
total_idx = np.load("total_idx_sim.npy")

total_features = total_features[total_y == 1]
total_transits = total_transits[total_y == 1]
total_params = total_params[total_y == 1]
total_SNR = total_SNR[total_y == 1]
total_idx = total_idx[total_y == 1]
total_y = total_y[total_y == 1]

header = './'

np.save(header + 'total_x_sim_true.npy', total_features)
np.save(header + 'total_y_sim_true.npy', total_y)
np.save(header + 'total_transits_sim_true.npy', total_transits)
np.save(header + 'total_params_sim_true.npy', total_params)
np.save(header + 'total_SNR_sim_true.npy', total_SNR)
np.save(header + 'total_idx_sim_true.npy', total_idx)


#!7z a all_files_true.zip total_x_sim_true.npy total_y_sim_true.npy total_transit_sim_true.npy total_params_sim_true.npy total_SNR_sim_true.npy total_idx_sim_true.npy
print('done')


total_features_train_true = total_features[:20000]
total_transits_train_true = total_transits[:20000]
total_y_train_true = total_y[:20000]
total_params_train_true = total_params[:20000]
total_SNR_train_true = total_SNR[:20000]

total_features_test_true = total_features[20000:30000]
total_transits_test_true = total_transits[20000:30000]
total_y_test_true = total_y[20000:30000]
total_params_test_true = total_params[20000:30000]
total_SNR_test_true = total_SNR[20000:30000]
print('done')

print('start')
print(total_features.shape, total_transits.shape, total_y.shape, total_params.shape, total_SNR.shape)

np.save(header + 'total_x_sim_train_true.npy', total_features_train_true)
np.save(header + 'total_y_sim_train_true.npy', total_y_train_true)
np.save(header + 'total_transit_sim_train_true.npy', total_transits_train_true)
np.save(header + 'total_params_sim_train_true.npy', total_params_train_true)
np.save(header + 'total_SNR_sim_train_true.npy', total_SNR_train_true)


np.save(header + 'total_x_sim_test_true.npy', total_features_test_true)
np.save(header + 'total_y_sim_test_true.npy', total_y_test_true)
np.save(header + 'total_transit_sim_test_true.npy', total_transits_test_true)
np.save(header + 'total_params_sim_test_true.npy', total_params_test_true)
np.save(header + 'total_SNR_sim_test_true.npy', total_SNR_test_true)


total_features = None
total_transits = None
total_y = None
total_params = None
total_SNR = None

total_features_train_true = None
total_transits_train_true = None
total_y_train_true = None
total_params_train_true = None
total_SNR_train_true = None

total_features_test_true = None
total_transits_test_true = None
total_y_test_true = None
total_params_test_true = None
total_SNR_test_true = None

#!7z a all_files_small_true.zip total_x_sim_train_true.npy total_y_sim_train_true.npy total_transit_sim_train_true.npy total_params_sim_train_true.npy total_SNR_sim_train_true.npy total_x_sim_test_true.npy total_y_sim_test_true.npy total_transit_sim_test_true.npy total_params_sim_test_true.npy total_SNR_sim_test_true.npy
print('done')


In [ ]:
#!rm -rf *_true.npy